## Using DeepLabV3 To The Steel Data

Source:
https://keras.io/examples/vision/deeplabv3_plus/

In [8]:
import os
import torch
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
# Import required packages 
import image_modeling   # import image_modeling.py file
import tensorflow_hub as hub
import datetime
# Load the TensorBoard notebook extension
%load_ext tensorboard



import torch
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
from PIL import Image
import glob
import random
import csv


import pathlib
import IPython.display as display
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
# Miscellaneous
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import os
import shutil
import cv2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 




In [10]:
# Let's make some dark cell magic. Why not!
from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
        print("Appended to file ", file)
    else:
        print('Written to file:', file)
    with open(file, mode) as f:
        f.write(cell.format(**globals()))        
    get_ipython().run_cell(cell)

In [14]:
%%write_and_run image_modeling.py

UsageError: %%write_and_run is a cell magic, but the cell body is empty.


In [15]:
#https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [16]:
path= os.getcwd()


train_path = path + '/Data/segmentation/train'
train_mask_path = path + '/Data/segmentation/train_mask'
test_path = path +'/Data/segmentation/test'
test_mask_path =path + '/Data/segmentation/train'
# Get the list of all files in directory tree at given path
train_list = getListOfFiles(train_path)
train_mask_list=getListOfFiles(train_mask_path)
test_list = getListOfFiles(test_path)
test_mask_list=getListOfFiles(test_mask_path)


In [17]:
# Import variables from image_modelling.py file
HEIGHT = image_modeling.HEIGHT
WIDTH = image_modeling.WIDTH
NCLASSES = image_modeling.NCLASSES
CLASS_NAMES = image_modeling.CLASS_NAMES
BATCH_SIZE = image_modeling.BATCH_SIZE
TRAINING_SIZE = len(train_list)
# Self coded
go_trough_per_epoch = int(1)
TRAINING_STEPS = (int(TRAINING_SIZE) // BATCH_SIZE) * go_trough_per_epoch


In [18]:
train_path_df = pd.DataFrame({'Path':train_list})
train_path_df.to_csv('./Data/train_path.csv') 

train_mask_path_df = pd.DataFrame({'Path':train_mask_list})
train_mask_path_df.to_csv('./Data/train_mask_path.csv')

test_path_df = pd.DataFrame({'Path':test_list})
test_path_df.to_csv('./Data/test_path.csv')

test_mask_path_df = pd.DataFrame({'Path':test_mask_list})
test_mask_path_df.to_csv('./Data/test_mask_path.csv')

In [19]:
# You can compare this output with the variables in the image_modelling.py file...
print(HEIGHT)
print(CLASS_NAMES)
print(NCLASSES)

256
['1', '2', '3', '4']
4


In [27]:
%%write_and_run -a image_modeling.py

# We set some parameters for the model
HEIGHT = 256 #image height
WIDTH = 1600 #image width
CHANNELS = 3 #image RGB channels
CLASS_NAMES = ['1', '2', '3', '4']
NCLASSES = len(CLASS_NAMES)
BATCH_SIZE = 32
SHUFFLE_BUFFER = 10 * BATCH_SIZE
AUTOTUNE = tf.data.experimental.AUTOTUNE

VALIDATION_SIZE = 370
VALIDATION_STEPS = VALIDATION_SIZE // BATCH_SIZE

Appended to file  image_modeling.py


In [28]:
%%write_and_run -a image_modeling.py

# Define the function that decodes in the images
def decode_image(image, reshape_dim):
    # JPEG is a compressed image format. So we want to 
    # convert this format to a numpy array we can compute with.
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    # 'decode_jpeg' returns a tensor of type uint8. We need for 
    # the model 32bit floats. Actually we want them to be in 
    # the [0,1] interval.
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Now we can resize to the desired size.
    image = tf.image.resize(image, reshape_dim)
    
    return image

Appended to file  image_modeling.py


In [25]:
train_list[0]

'/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/Data/segmentation/train/c1/eeffa4c49.jpg'

In [29]:
# Let us test our decoding function.
img = tf.io.read_file(train_list[0])    

# TODO: take the function above and decode the image
img = decode_image(img, [256,1600])

In [30]:
%%write_and_run -a image_modeling.py

# The train set actually gives only the paths to the training images.
# We want to create a dataset of training images, so we need a 
# function that can handle this for us.
def decode_dataset(data_row):
    record_defaults = ['path', 'class']
    filename, label_string = tf.io.decode_csv(data_row, record_defaults)
    image_bytes = tf.io.read_file(filename=filename)
    label = tf.math.equal(label_string, CLASS_NAMES)
    return image_bytes, label

Appended to file  image_modeling.py


In [31]:
dataset = tf.data.TextLineDataset('./Data/train_path.csv')
it = iter(dataset)
record_defaults = ['path','class'] # defines dtype
# output dtype of decode_csv will be two strings. could have written ['chicken','egg'] with same outcome. But not e.g. [1,'class'].
filename, label_string = tf.io.decode_csv(next(it), record_defaults)
filename

2022-02-16 21:03:32.002318: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<tf.Tensor: shape=(), dtype=string, numpy=b'path'>

In [32]:
%%write_and_run -a image_modeling.py

# Next we construct a function for pre-processing the images.
def read_and_preprocess(image_bytes, label, augment_randomly=False):
    if augment_randomly: 
        image = decode_image(image_bytes, [HEIGHT + 8, WIDTH + 8])
        # TODO: Augment the image.
        import random
        offset_h = random.randint(0,8)
        offset_w = random.randint(0,8)
        image = image [offset_h:264-(8-offset_h),offset_w:1608-(8-offset_w)]
    else:
        image = decode_image(image_bytes, [HEIGHT, WIDTH])
    return image, label

def read_and_preprocess_with_augmentation(image_bytes, label): 
    return read_and_preprocess(image_bytes, label, augment_randomly=True)

Appended to file  image_modeling.py


In [33]:
%%write_and_run -a image_modeling.py

# Now we can create the dataset.
def load_dataset(file_of_filenames, batch_size, training=False):
    # We create a TensorFlow Dataset from the list of files.
    # This dataset does not load the data into memory, but instead
    # pulls batches one after another.
    dataset = tf.data.TextLineDataset(filenames=file_of_filenames).\
        map(decode_dataset)
    
    if training:
        # TODO: Use augmentation here.
        dataset = dataset.map(read_and_preprocess_with_augmentation).\
            shuffle(SHUFFLE_BUFFER).\
            repeat(count=None) # Infinite iterations
    else: 
        # Evaluation or testing
        dataset = dataset.map(read_and_preprocess).\
            repeat(count=1) # One iteration
            
    # The dataset will produce batches of BATCH_SIZE and will
    # automatically prepare an optimized number of batches while the prior one is
    # trained on.
    return dataset.batch(batch_size).prefetch(buffer_size=AUTOTUNE)



Appended to file  image_modeling.py


In [47]:
dataset = tf.data.TextLineDataset(filenames='./Data/train_path.csv').\
        map(decode_dataset)

dataset = dataset.map(read_and_preprocess).\
            repeat(count=1) # One iteration

dataset.batch(32).prefetch(buffer_size=AUTOTUNE)

<PrefetchDataset shapes: ((None, 256, 1600, 3), (None, 4)), types: (tf.float32, tf.bool)>

In [40]:
# Let us see, if data loading works as intended.
#train_path = './Data/train_path.csv'
train_data = load_dataset('./Data/train_path.csv', 1)
# Create an iterator that runs over the training dataset.
it = iter(train_data)
next(it)

2022-02-16 21:05:51.965674: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 0; No such file or directory
2022-02-16 21:05:51.965696: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 3; No such file or directory
2022-02-16 21:05:51.965712: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 1; No such file or directory
2022-02-16 21:05:51.965744: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 7; No such file or directory
2022-02-16 21:05:51.965769: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 6; No such file or directory
2022-02-16 21:05:51.965784: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 2; No such file or directory
2022

NotFoundError: path; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]

2022-02-16 21:04:58.131455: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 0; No such file or directory
2022-02-16 21:04:58.131514: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 3; No such file or directory
2022-02-16 21:04:58.131548: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 5; No such file or directory
2022-02-16 21:04:58.131588: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 4; No such file or directory
2022-02-16 21:04:58.131615: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 1; No such file or directory
2022-02-16 21:04:58.131653: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 6; No such file or directory
2022

NotFoundError: path; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]

In [48]:
# Iterate and see the pictures and labels
img_batch, labels = next(it)
image = img_batch[0]
plt.imshow(image)
print(labels[0])

2022-02-16 21:10:35.353762: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: 17; No such file or directory


NotFoundError: 0; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:

def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

NUM_CLASSES = 4
IMAGE_SIZE = 128


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

In [ ]:
# Iterate and see the pictures and labels
#train_img_x, train_mask_y =next(my_train)
#test_img_x, test_mask_y =next(my_test)

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=["accuracy"],
)


history = model.fit(train, validation_data= test, epochs=2, verbose=1)

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()